In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm


In [2]:
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/dataurb/datacat010.csv'


Mounted at /content/drive


In [3]:
df = pd.read_csv(file_path)

In [4]:
df['Сообщение'] = df['Сообщение'].astype(str)
df['Категория'] = df['Категория'].astype(str).str.strip()

print("Уникальные категории после предобработки:", df['Категория'].unique())
print("Количество уникальных категорий:", len(df['Категория'].unique()))


Уникальные категории после предобработки: ['Валка сухих/аварийных деревьев' 'Восстановление после разрытий'
 'Вывоз валки' 'Вывоз ТБО (либо уборка после)' 'Другие вопросы'
 'Жалобы на объекты предпринимательства' 'Консультация по компетенции'
 'Не проблема' 'Общественная безопастность' 'Освещение не работает'
 'Отключение электричества' 'Открытый люк на дороге' 'Отопление'
 'Ремонт детских элементов' 'Ремонт и содержание автополива'
 'Санитарная обрезка' 'Уборка улиц (сан. очистка)']
Количество уникальных категорий: 17


In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Сообщение'].tolist(),
    df['Категория'].tolist(),
    test_size=0.2,
    random_state=42
)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

category_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Category Mapping:", category_mapping)

Category Mapping: {'Валка сухих/аварийных деревьев': 0, 'Восстановление после разрытий': 1, 'Вывоз ТБО (либо уборка после)': 2, 'Вывоз валки': 3, 'Другие вопросы': 4, 'Жалобы на объекты предпринимательства': 5, 'Консультация по компетенции': 6, 'Не проблема': 7, 'Общественная безопастность': 8, 'Освещение не работает': 9, 'Отключение электричества': 10, 'Открытый люк на дороге': 11, 'Отопление': 12, 'Ремонт детских элементов': 13, 'Ремонт и содержание автополива': 14, 'Санитарная обрезка': 15, 'Уборка улиц (сан. очистка)': 16}


In [6]:
model_name = "ai-forever/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(train_labels)))

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
class RubertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RubertDataset(train_encodings, train_labels)
val_dataset = RubertDataset(val_encodings, val_labels)

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [9]:
from transformers import TrainingArguments, Trainer
import torch

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="f1",
    greater_is_better=True,
    run_name="gofropartner"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.010100,0.240218,0.954383,0.951639,0.953261,0.954383
2,0.076800,0.225519,0.962224,0.961562,0.962261,0.962224
3,0.001100,0.232319,0.964362,0.963889,0.964779,0.964362
4,0.007300,0.257839,0.965075,0.965186,0.966046,0.965075
5,0.000400,0.247387,0.968639,0.968746,0.969375,0.968639
6,0.000300,0.260818,0.969351,0.969532,0.970218,0.969351


TrainOutput(global_step=8418, training_loss=0.15418172096040889, metrics={'train_runtime': 3972.2533, 'train_samples_per_second': 8.474, 'train_steps_per_second': 2.119, 'total_flos': 7842599879685120.0, 'train_loss': 0.15418172096040889, 'epoch': 6.0})

In [11]:
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 0.2608177661895752, 'eval_accuracy': 0.9693513898788311, 'eval_f1': 0.9695319048154992, 'eval_precision': 0.9702181915440584, 'eval_recall': 0.9693513898788311, 'eval_runtime': 37.0704, 'eval_samples_per_second': 37.847, 'eval_steps_per_second': 9.468, 'epoch': 6.0}


In [12]:
model.save_pretrained('./model9ep')
tokenizer.save_pretrained('./model9ep')
label_encoder.classes_.dump('./model9ep/label_classes.pkl')

In [13]:
import shutil
shutil.make_archive("model9ep", 'zip', "model9ep")

'/content/model9ep.zip'

In [14]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/model9ep.zip /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
